<a href="https://colab.research.google.com/github/Vitecnico/Projeto-Estudos-Data-Engineer/blob/main/Pyspark_Analises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação de bibliotecas


In [ ]:
!pip install pyspark

In [ ]:
!pip install gcsfs

#Import

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from google.cloud import storage
import os
import pandas as pd

#Chave de Segurança Google

In [ ]:
serviceAccount = '/content/aulas-bc26-laisa-510383ea5245.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
pd.set_option('display.max_columns',100)

#Importação e extração de datasets

##Cloud Storage/Bucket

In [ ]:
#FAZER O ACESSO A BUCKET
client = storage.Client()

#CRIAR UMA VARIÁVEL PARA RECEBER O NOME DA BUCKET

bucket = client.get_bucket('projetodupladinamica')

#Atualiza/Cria um arquivo no bucket a partir de um arquivo local
#blob1 = bucket.blob('D.SDA.PDA.005.CAT.202201')
#blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202201.csv')

#ESCOLHER O ARQUIVO DENTRO DA BUCKET


path = 'gs://projetodupladinamica/INSS2022.csv'
path2 = 'gs://projetodupladinamica/CAT.202201.orig.csv'
#path2 = 'gs://projetodupladinamica/CAT.202204.orig.csv'
#path2 = 'gs://projetodupladinamica/CAT.202210.orig.csv'

##Spark Session

In [ ]:
spark = (
    SparkSession.builder
                .master('local')
                .appName('teste')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

In [ ]:
spark

#Dataframes

In [ ]:
#criando dataframes
df_concatenado = (spark.read.format('csv')
                 .option('header','true')
                 .option('inferschema', 'true')
                 .option('delimiter',',')
                 .load(path)    
)
df = (spark.read.format('csv')
                            .option('header','true')
                            .option('inferschema', 'true')
                            .option('delimiter',';')
                            .load(path2)    
)

In [ ]:
df_concatenado = df_concatenado.drop(F.col('_c0'))

# Formatando df originais para mongo db

In [ ]:
# Renomeando as colunas
df_for_mongo = (df_concatenado.withColumnRenamed('CNAE2.0 Empregador5','CNAE2_empregador')
               .withColumnRenamed('Data Acidente','data_acidente')
               .withColumnRenamed('Data Acidente1','data_acidente')
               .withColumnRenamed('CNAE2.0 Empregador4','CNAE2_empregador_Num')
               .withColumnRenamed('Emitente CAT','emitente_CAT')
               .withColumnRenamed('Espécie do benefício','especie_do_beneficio')
               .withColumnRenamed('Filiação Segurado','filiacao_segurado')
               .withColumnRenamed('Indica Óbito Acidente','indica_obito_acidente')
               .withColumnRenamed('Munic Empr','municipio_empresa')
               .withColumnRenamed('Natureza da Lesão','natureza_da_lesao')
               .withColumnRenamed('Origem de Cadastramento CAT','origem_de_cadastramento_CAT')
               .withColumnRenamed('Parte Corpo Atingida','parte_corpo_atingida')
               .withColumnRenamed('Sexo','sexo')
               .withColumnRenamed('Tipo do Acidente','tipo_do_acidente')
               .withColumnRenamed('UF  Munic.  Acidente','UF_municipio_acidente')
               .withColumnRenamed('UF Munic. Empregador','UF_municipio_empregador')
               .withColumnRenamed('Data Acidente18','data_acidente1')
               .withColumnRenamed('Data Despacho Benefício','data_despacho_beneficio')
               .withColumnRenamed('Data Acidente20','data_acidente2')
               .withColumnRenamed('Data Nascimento','data_nascimento')
               .withColumnRenamed('Data Emissão CAT','data_emissao_CAT')
               .withColumnRenamed('CNPJ/CEI Empregador','CNPJ/CEI_empregador')
               .withColumnRenamed('Data Acidente.1','data_acidente1')
               .withColumnRenamed('Data Acidente.2','data_acidente2')
               .withColumnRenamed('CNAE2.0 Empregador','CNAE2_empregador_Num')
               .withColumnRenamed('CNAE2.0 Empregador.1','CNAE2_empregador')
               .withColumnRenamed('Sexo','sexo')
               .withColumnRenamed('Filia��o Segurado','filiacao_segurado')
               .withColumnRenamed('Esp�cie do benef�cio','especie_do_beneficio')
               .withColumnRenamed('Indica �bito Acidente','indica_obito_acidente')
               .withColumnRenamed('Natureza da Les�o','natureza_da_lesao')
               .withColumnRenamed('Data Despacho Benef�cio','data_despacho_beneficio')
               .withColumnRenamed('Data Emiss�o CAT','data_emissao_CAT')
               .withColumnRenamed('Agente  Causador  Acidente','Agente_Causador_Acidente')
               

)
df_for_mongo.dtypes

[('Agente_Causador_Acidente', 'string'),
 ('data_acidente', 'string'),
 ('CBO', 'string'),
 ('CID-10', 'string'),
 ('CNAE2_empregador_Num', 'int'),
 ('CNAE2_empregador', 'string'),
 ('emitente_CAT', 'string'),
 ('especie_do_beneficio', 'string'),
 ('filiacao_segurado', 'string'),
 ('indica_obito_acidente', 'string'),
 ('municipio_empresa', 'string'),
 ('natureza_da_lesao', 'string'),
 ('origem_de_cadastramento_CAT', 'string'),
 ('parte_corpo_atingida', 'string'),
 ('sexo', 'string'),
 ('tipo_do_acidente', 'string'),
 ('UF_municipio_acidente', 'string'),
 ('UF_municipio_empregador', 'string'),
 ('data_acidente1', 'string'),
 ('data_despacho_beneficio', 'string'),
 ('data_acidente2', 'string'),
 ('data_nascimento', 'string'),
 ('data_emissao_CAT', 'string'),
 ('CNPJ/CEI_empregador', 'string')]

In [ ]:
#drop de coluna repetida
df_for_mongo2 = df_for_mongo.drop(F.col('data_acidente1'))

In [ ]:
df_for_mongo2 = df_for_mongo2.dropDuplicates()

In [ ]:
df_for_mongo2.show()

+------------------------+-------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------------------+--------------------+--------------------+---------------------------+--------------------+-------------+----------------+---------------------+-----------------------+-----------------------+--------------+---------------+----------------+-------------------+
|Agente_Causador_Acidente|data_acidente|                 CBO|              CID-10|CNAE2_empregador_Num|    CNAE2_empregador|       emitente_CAT|especie_do_beneficio|   filiacao_segurado|indica_obito_acidente|   municipio_empresa|   natureza_da_lesao|origem_de_cadastramento_CAT|parte_corpo_atingida|         sexo|tipo_do_acidente|UF_municipio_acidente|UF_municipio_empregador|data_despacho_beneficio|data_acidente2|data_nascimento|data_emissao_CAT|CNPJ/CEI_empregador|
+------------------------+-------------+--------------------+-

In [ ]:
df_for_mongo_pd = df_for_mongo2.toPandas()


In [ ]:
df_for_mongo_pd.to_csv('dfmongo3.csv', index=False)

In [ ]:
#blob = bucket.blob('CAT.202201.ToMongo3.csv')
#blob.upload_from_filename('/content/dfmongo3.csv')

# analises do pd


In [ ]:
pd.unique(df_for_mongo_pd['tipo_do_acidente'])

array(['Doen�a  ', 'T�pico  ', 'Trajeto ', 'Ignorado'], dtype=object)

In [ ]:
df_for_mongo_pd.dtypes

Agente_Causador_Acidente       object
data_acidente                  object
CBO                            object
CID-10                         object
CNAE2_empregador_Num            int32
CNAE2_empregador               object
emitente_CAT                   object
especie_do_beneficio           object
filiacao_segurado              object
indica_obito_acidente          object
municipio_empresa              object
natureza_da_lesao              object
origem_de_cadastramento_CAT    object
parte_corpo_atingida           object
sexo                           object
tipo_do_acidente               object
UF_municipio_acidente          object
UF_municipio_empregador        object
data_despacho_beneficio        object
data_acidente2                 object
data_nascimento                object
data_emissao_CAT               object
CNPJ/CEI_empregador            object
dtype: object

In [ ]:
df_for_mongo_pd.isnull().sum()

Agente_Causador_Acidente       0
data_acidente                  0
CBO                            0
CID-10                         0
CNAE2_empregador_Num           0
CNAE2_empregador               0
emitente_CAT                   0
especie_do_beneficio           0
filiacao_segurado              0
indica_obito_acidente          0
municipio_empresa              0
natureza_da_lesao              0
origem_de_cadastramento_CAT    0
parte_corpo_atingida           0
sexo                           0
tipo_do_acidente               0
UF_municipio_acidente          0
UF_municipio_empregador        0
data_despacho_beneficio        0
data_acidente2                 0
data_nascimento                0
data_emissao_CAT               0
CNPJ/CEI_empregador            0
dtype: int64

## tratando valores

In [ ]:
colunas = df_for_mongo2.columns
print(colunas)

['Agente_Causador_Acidente', 'data_acidente', 'CBO', 'CID-10', 'CNAE2_empregador_Num', 'CNAE2_empregador', 'emitente_CAT', 'especie_do_beneficio', 'filiacao_segurado', 'indica_obito_acidente', 'municipio_empresa', 'natureza_da_lesao', 'origem_de_cadastramento_CAT', 'parte_corpo_atingida', 'sexo', 'tipo_do_acidente', 'UF_municipio_acidente', 'UF_municipio_empregador', 'data_despacho_beneficio', 'data_acidente2', 'data_nascimento', 'data_emissao_CAT', 'CNPJ/CEI_empregador']


In [ ]:
df_for_mongo3 = df_for_mongo2

In [ ]:
for i in colunas:
    df_for_mongo3 = df_for_mongo3.withColumn(i,F.when((F.col(i).like('%}%')) ,'N/A').otherwise(F.col(i))) \
                                 .withColumn(i,F.when((F.col(i).like('%Nulo%')),'N/A').otherwise(F.col(i))) \
                                 .withColumn(i,F.when((F.col(i)=='00.000.000.000.000'),'N/A').otherwise(F.col(i))) \
                                 .withColumn(i,F.when((F.col(i)=='0000/00'),'N/A').otherwise(F.col(i)))\
                                 

In [ ]:
df_for_mongo3.show()

+------------------------+-------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------------------+--------------------+--------------------+---------------------------+--------------------+-------------+----------------+---------------------+-----------------------+-----------------------+--------------+---------------+----------------+-------------------+
|Agente_Causador_Acidente|data_acidente|                 CBO|              CID-10|CNAE2_empregador_Num|    CNAE2_empregador|       emitente_CAT|especie_do_beneficio|   filiacao_segurado|indica_obito_acidente|   municipio_empresa|   natureza_da_lesao|origem_de_cadastramento_CAT|parte_corpo_atingida|         sexo|tipo_do_acidente|UF_municipio_acidente|UF_municipio_empregador|data_despacho_beneficio|data_acidente2|data_nascimento|data_emissao_CAT|CNPJ/CEI_empregador|
+------------------------+-------------+--------------------+-

# Analises


In [ ]:

len(df.columns) == len(df_concatenado.columns)

In [ ]:
#dfpd = df.toPandas()

In [ ]:
#dfpd

In [ ]:
df.show(truncate= False)


In [ ]:
df_concatenado.show(truncate= False)

In [ ]:
df_count = df_concatenado.groupBy("Data Despacho Benefício").count().filter("count > 1")
df_count.orderBy(F.col('count').desc()).show()

In [ ]:
df2=df.groupBy("CBO").count().filter("count > 1")
##df4=df3.groupBy("fala").count()
df2.orderBy(F.col('count').desc()).show()

In [ ]:
df.filter(F.col('CBO').like('%}%')).show(truncate= False)

In [ ]:
df1 = df_concatenado
#df1 = df_concatenado.withColumn('agente', F.split(df_concatenado['Agente  Causador  Acidente'], ',').getItem(0)) \
#                    .withColumn('causador', F.split(df_concatenado['Agente  Causador  Acidente'], ',').getItem(1)) \
#                    .withColumn('acidente', F.split(df_concatenado['Agente  Causador  Acidente'], ',').getItem(2))
df1.show(truncate=False)

In [ ]:
dfrename = (df1.withColumnRenamed('CNAE2.0 Empregador5','id')
               .withColumnRenamed('Data Acidente','data_acidente')
               .withColumnRenamed('Data Acidente1','data_acidente1')
               .withColumnRenamed('CNAE2.0 Empregador4','CNAE2_empregador_Num')
               .withColumnRenamed('CNAE2.0 Empregador5','CNAE2_empregador')
               .withColumnRenamed('Emitente CAT','emitente_CAT')
               .withColumnRenamed('Espécie do benefício','especie_do_beneficio')
               .withColumnRenamed('Filiação Segurado','filiacao_segurado')
               .withColumnRenamed('Indica Óbito Acidente','indica_obito_acidente')
               .withColumnRenamed('Munic Empr','municipio_empresa')
               .withColumnRenamed('Natureza da Lesão','natureza_da_lesao')
               .withColumnRenamed('Origem de Cadastramento CAT','origem_de_cadastramento_CAT')
               .withColumnRenamed('Parte Corpo Atingida','parte_corpo_atingida')
               .withColumnRenamed('Sexo','sexo')
               .withColumnRenamed('Tipo do Acidente','tipo_do_acidente')
               .withColumnRenamed('UF  Munic.  Acidente','UF_municipio_acidente')
               .withColumnRenamed('UF Munic. Empregador','UF_municipio_empregador')
               .withColumnRenamed('Data Acidente18','data_acidente18')
               .withColumnRenamed('Data Despacho Benefício','data_despacho_beneficio')
               .withColumnRenamed('Data Acidente20','data_acidente20')
               .withColumnRenamed('Data Nascimento','data_nascimento')
               .withColumnRenamed('Data Emissão CAT','data_emissao_CAT')
               .withColumnRenamed('CNPJ/CEI Empregador','CNPJ/CEI_empregador')
               .withColumnRenamed('Data Acidente.1','data_acidente1')
               .withColumnRenamed('Data Acidente.2','data_acidente2')
               .withColumnRenamed('CNAE2.0 Empregador','CNAE2_empregador_Num')
               .withColumnRenamed('CNAE2.0 Empregador.1','CNAE2_empregador')
               .withColumnRenamed('Sexo','sexo')

)
dfrename.dtypes

# Filtro e Junção


In [ ]:
dfrename1 = dfrename.drop(F.col('data_acidente1'))
#dfrename1 = dfrename1.drop(F.col('Agente  Causador  Acidente'))
dfrename1 = dfrename1.drop(F.col('_c0'))
#dfrename.select(['Data Acidente.1']).distinct().show()

In [ ]:
dfrename1 = dfrename1.dropDuplicates()

In [ ]:
dfrename1.show()

In [ ]:
dfvalor=dfrename1.groupBy("sexo").count().filter("count > 1")
dfvalor.orderBy(F.col('count').desc()).show()

In [ ]:
( dfrename1.withColumn('id_tipo_acidente', 
                                F.when(F.col('tipo_do_acidente').like('%Típico%'),F.lit(1))
                                .when(F.col('tipo_do_acidente').like('%Ignorado%'),F.lit(2))
                                .when(F.col('tipo_do_acidente').like('%Trajeto%'),F.lit(3))
                                .otherwise(F.lit(4)))
                                .show()
)

In [ ]:
( dfrename1.withColumn('id_tipo_acidente', 
                                F.when(F.col('tipo_do_acidente').like('%Típico%'),F.lit(1))
                                .when(F.col('tipo_do_acidente').like('%Ignorado%'),F.lit(2))
                                .when(F.col('tipo_do_acidente').like('%Trajeto%'),F.lit(3))
                                .otherwise(F.lit(4)))
                                .show()
)

In [ ]:
#print(df_renameded.count(),dfrename1.count())

In [ ]:
dfselect = dfrename1.select(F.col('data_acidente'), F.col('CNAE2_empregador'), F.col('indica_obito_acidente'),F.col('emitente_CAT'), F.col('tipo_do_acidente')).filter((F.col('indica_obito_acidente') == 'Sim') & (F.col('tipo_do_acidente').like('%Doença%') ))
dfselect.orderBy(F.col('data_acidente').desc()).show()

#outros testes

In [ ]:
#dfrename = dfrename.na.drop(subset=["id"])


In [ ]:
#dfrename.write.csv("/content/teste/test.csv")x

In [ ]:
dfrename3 = dfrename.withColumn('data_acidente3_mes', F.split(dfrename['data_acidente2'], '/').getItem(1)) \
                    .withColumn('data_acidente3_ano', F.split(dfrename['data_acidente2'], '/').getItem(2)) \
                    .withColumn('data_acidente4_mes', F.split(dfrename['data_acidente'], '/').getItem(1)) \
                    .withColumn('data_acidente4_ano', F.split(dfrename['data_acidente'], '/').getItem(0)) 
                    

In [ ]:
dfrename3.show()

In [ ]:
d5 = dfrename3.select(F.col('data_acidente3_ano'), F.col('data_acidente4_ano')).filter(F.col('data_acidente3_ano') !=  F.col('data_acidente4_ano'))
d5.show()

In [ ]:
d6 = dfrename.select(F.col('data_acidente'), F.col('data_acidente1'))
d5 = dfrename.select(F.col('data_acidente'), F.col('data_acidente2')).filter(F.col('data_acidente') ==  F.col('data_acidente2'))
d5.show()
print(d5.count())
print(d6.count())

In [ ]:
dfrename1.dtypes